# Optimization Sample: Shipping

This notebook follows a sample shipping company that has a difficult business problem: balancing the loads of container ships at port. We'll apply the power of QIO to generate a solution to a simplified version of this problem.

## Pre-requisites

1. [Create an Azure Quantum Workspace](https://dev.azure.com/AzureQuantum-PreviewCustomers/PrivatePreview/_wiki/wikis/Azure%20Quantum%20Documentation/36/Creating-an-Azure-Quantum-Workspace)
2. Install the `azure-quantum` Python module by following the [private preview instructions](https://dev.azure.com/AzureQuantum-PreviewCustomers/PrivatePreview/_wiki/wikis/Azure%20Quantum%20Documentation/37/Using-the-Python-SDK).


## The Problem
Contoso Logistics Inc is a world class freight company operating a fleet of large container ships. When these ships are docked at port Contoso Logistics needs to distribute containers between the two ships as evenly as possible. If they make one ship significantly heavier than the other then it will move slower and consume more fuel - delaying shipments and costing a significant amount of money! To make things harder, the weights of individual containers can vary greatly so it's not easy to divide them.

This problem is known as the Number partitioning problem and is NP-complete. It is however relatively straightforward to apply Quantum Inspired Optimization (QIO) to it to generate a good solution. Let's explore how.

## What does it mean to Optimize?
We're using Quantum-Inspired Optimization to solve this problem - but what does that actually mean?

In our case, optimize means "to find a solution with minimal cost" - which is exactly what the Solvers in Azure Quantum do. They take a representation of a difficult problem and apply techniques from physics to find a solution with the least cost.

> Note that solvers can be applied to very difficult problems where it's infeasible to confirm whether the solution found is the best solution that exists. So keep in mind that there may be better solutions than those returned by the solvers.

The solvers in Azure Quantum expect a `Binary Optimization Problem`, which is a problem expressed in the following (simplified) format:
$$  H = \Large\sum_{i} w_{i}x_{i} + \sum_{i,j} w_{i,j}x_{i}x_{j} $$
$$
x_{i} \in \left\{
        \begin{array}{ll}
            0 & \quad \\
            1 & \quad
        \end{array}
    \right.
\text{or}\quad
x_{i} \in \left\{
        \begin{array}{ll}
            1 & \quad \\
            -1 & \quad
        \end{array}
    \right.
$$

That is, a summation that is composed of weights ${w}$, and binary variables ${x_i}$. Let's explore how you formulate such a problem.


## Understanding the Problem
In the number partitioning problem, we have a set ${W}$ of container weights which we would like to partition into two sets: ${W_a}$ and ${W_b}$ (containers on the ships, ${a}$ and ${b}$). In this section, our goal will be to develop a representation of the problem that we can provide to the QIO solver.

## Breaking down the Problem
Let's start by coming up with an equation for the weight of a given ship, which is the sum of all the containers on the ship. This is expressed in the below equation, where ${w_i}$ is the weight of container ${i}$:

$$ \Large\sum_{i} w_{i} $$

Ideally, we'd like a solution where the weight difference between the ships is as small as possible. This is expressed by the following expression:

$$ H = \Large\sum_{a} w_{i} - \Large\sum_{b} w_{i} $$

If the value of ${H}$ is zero, we know the ships are equally loaded.

Next, we'll introduce a variable, ${x_i}$, to represent whether an individual container ${i}$ is assigned to ship ${a}$ or ship ${b}$. Because we can assign the container ${i}$ to either ship, the variable ${x_i}$ can be take on two different values - which makes it a `binary` variable. For convenience, we'll say the two values it can take on are ${1}$ and ${-1}$. ${1}$ will represent that the container is placed on ship ${a}$, and ${-1}$ will represent that the container is placed on ship ${b}$.

> Because of our choice to make ${i}$ be either ${1}$ or ${-1}$ this type of problem is called an `Ising` problem.

By introducing this variable ${x_i}$ to the previous equation, it can be simplified to:

$$ H = \Large\sum_{i} w_{i} x_{i} $$

The function ${H}$ will be called our `cost function` as it describes the cost of a given solution.

> The letter ${H}$ is traditionally used to represent a cost function and is also referred to as a `Hamiltonian` in a nod towards the quantum mechanical roots of Quantum-Inspired Optimization techniques.

## The final model
There's one last change we need to make before we can solve our problem. If we look at our cost function ${H}$ there's a flaw: the solution with the _least_ cost is to simply assign all containers to ship ${b}$ by setting all ${x_i}={-1}$ - that's not right! To fix this we'll take a simple step - we'll square the right hand side of the equation so that it cannot be negative:

$$ H = \Large(\sum_{i} w_{i} x_{i})^{2} $$

This is somewhat arbitrary, but it yields a cost function with the right properties:
- If all the containers are on one ship, the function is at its highest value - reflecting that this is the least optimal solution
- If the containers are perfectly balanced, the value of the summation inside the square is ${0}$ - the function is at its lowest value
- In this case, we don't care about the actual value of ${H}$, just that it's as small as possible.

Next, let's express this problem in Python and solve it for a few cases.


## Solving the Problem in Python

First, we must instantiate a `Workspace` object which allows you to connect to the Workspace you've previously deployed in Azure.

In [ ]:
from azure.quantum import Workspace

# Copy the settings for your workspace below
workspace = Workspace(
    subscription_id=    "", # add your subscription_id
    resource_group=     "", # add your resource_group
    name=               "", # add your workspace name
    storage=            "", # add your storage connection string
)

workspace.login()

Next, we'll define a function that takes an array of container weights and returns a `Problem` object that represents the cost function.

In [ ]:
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term

def createProblemForContainerWeights(containerWeights: List[int]) -> Problem:
    terms: List[Term] = []

    # Expand the squared summation
    for i in range(len(containerWeights)):
        for j in range(len(containerWeights)):
            if i == j:
                # Skip the terms where i == j as they form constant terms in an Ising problem and can be disregarded:
                # w_i∗w_j∗x_i∗x_j = w_i​*w_j∗(x_i)^2 = w_i∗w_j​​
                # for x_i = x_j, x_i ∈ {1, -1}
                continue
            
            terms.append(
                Term(
                    w = containerWeights[i] * containerWeights[j],
                    indices = [i, j]
                )
            )

    # Return an Ising-type problem
    return Problem(name="Ship Sample Problem", problem_type=ProblemType.ising, terms=terms)

Next, we'll define the list of containers and their weights and instantiate a problem:

In [ ]:
# This array contains a list of the weights of the containers
containerWeights = [1, 5, 9, 21, 35, 5, 3, 5, 10, 11]

# Create a problem for the list of containers:
problem = createProblemForContainerWeights(containerWeights)

Now, submit it to Azure using the ParallelTempering solver:

> We'll use Parameter-Free Parallel Tempering with a timeout of 100 seconds. Solver selection and tuning is beyond the scope of this tutorial.

In [ ]:
from azure.quantum.optimization import ParallelTempering
import time

# Instantiate a solver to solve the problem. 
solver = ParallelTempering(workspace, timeout=100)

# Optimize the problem
print('Submitting problem...')
start = time.time()
result = solver.optimize(problem)
timeElapsed = time.time() - start
print(f'Result in {timeElapsed} seconds: ', result)

Lastly, print out a summary of what the solution means:

In [ ]:
def printResultSummary(result):
    # Print a summary of the result
    shipAWeight = 0
    shipBWeight = 0
    for container in range(len(result['solution'])):
        containerAssignment = result['solution'][container]
        containerWeight = containerWeights[container]
        ship = ''
        if containerAssignment == 1:
            ship = 'A'
            shipAWeight += containerWeight
        else:
            ship = 'B'
            shipBWeight += containerWeight

        print(f'Container {container} with weight {containerWeight} was placed on Ship {ship}')

    print(f'\nTotal weights: \n\tShip A: {shipAWeight} tonnes \n\tShip B: {shipBWeight} tonnes')

printResultSummary(result)

## Improving the Cost Function
The cost function we've built works well so far, but let's take a closer look at the `Problem` that was generated:


In [ ]:
print(f'The problem has {len(problem.terms)} terms for {len(containerWeights)} containers:')
print(problem.terms)

That's alot of terms for just 10 containers! On closer inspection however, you'll note that there are essentially duplicated terms that result from having squared the right hand side of the equation. For example, look at the last term: `{'w': 110, 'ids': [9, 8]}`. If you look through the rest of the terms, you'll find a symmetrical copy of this term: `{'w': 110, 'ids': [8, 9]}`.

This duplicate encodes the exact same information in our cost function. However, because we don't actually care about the value of the cost function (just the shape), we can omit these terms too by a slight modification to our cost function:

$$ H = \Large(\sum_{i<j} w_{i} x_{i})^2 $$

In code, this means a small modification to the `createProblemForContainerWeights` function:

In [ ]:
def createSimplifiedProblemForContainerWeights(containerWeights: List[int]) -> Problem:
    terms: List[Term] = []

    # Expand the squared summation
    for i in range(len(containerWeights)-1):
        for j in range(i+1, len(containerWeights)):
            terms.append(
                Term(
                    w = containerWeights[i] * containerWeights[j],
                    indices = [i, j]
                )
            )

    # Return an Ising-type problem
    return Problem(name="Ship Sample Problem (Simplified)", problem_type=ProblemType.ising, terms=terms)

Let's check that this creates a smaller problem:

In [ ]:
# Create the simplified problem
simplifiedProblem = createSimplifiedProblemForContainerWeights(containerWeights)
print(f'The simplified problem has {len(simplifiedProblem.terms)} terms')

Success! The problem has half as many terms. Now let's run it and verify the result:

In [ ]:
# Optimize the problem
print('Submitting simplified problem...')
start = time.time()
simplifiedResult = solver.optimize(simplifiedProblem)
timeElapsedSimplified = time.time() - start
print(f'Result in {timeElapsedSimplified} seconds: ', simplifiedResult)
printResultSummary(simplifiedResult)


As you can see, the quality of the solution is the same for both cost functions - the ships are loaded within 1 tonne of each other (a perfect solution does not exist). This reveals an important fact about using QIO solvers: it is often possible (and neccesary) to optimize the cost function in order to generate more optimal solutions more quickly.